In [8]:
import random
import json

import torch

from ipynb.fs.full.model import NeuralNet
from ipynb.fs.full.nltk_utils import tokenize
from ipynb.fs.full.nltk_utils import bag_of_words

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

with open(r"C:\Users\lakshay.chopra\Chatbot GUI\intents.json") as f:
    intents = json.load(f)
    print(intents)  # Print the loaded JSON data


FILE = "data.pth"
data = torch.load(FILE)

input_size = data["input_size"]
hidden_size = data["hidden_size"]
output_size = data["output_size"]
all_words = data['all_words']
tags = data['tags']
model_state = data["model_state"]

model = NeuralNet(input_size, hidden_size, output_size).to(device)
model.load_state_dict(model_state)
model.eval()

bot_name = "Sam"

def get_response(msg):
    sentence = tokenize(msg)
    X = bag_of_words(sentence, all_words)
    X = X.reshape(1, X.shape[0])
    X = torch.from_numpy(X).to(device)

    output = model(X)
    _, predicted = torch.max(output, dim=1)

    tag = tags[predicted.item()]

    probs = torch.softmax(output, dim=1)
    prob = probs[0][predicted.item()]
    if prob.item() > 0.75:
        for intent in intents['intents']:
            if tag == intent["tag"]:
                return random.choice(intent['responses'])
    
    return "I do not understand..."

    

{'intents': [{'tag': 'greeting', 'patterns': ['Hi', 'Hey', 'Hello'], 'responses': ['Hello!', 'Hi there!']}, {'tag': 'goodbye', 'patterns': ['Goodbye', 'See you later'], 'responses': ['Goodbye!', 'See you later, thanks for visiting', 'Have a nice day', 'Bye! Come back again soon.']}, {'tag': 'thanks', 'patterns': ['Thanks', 'Thank you', "That's helpful", "Thank's a lot!"], 'responses': ['Happy to help!', 'Any time!', 'My pleasure']}, {'tag': 'items', 'patterns': ['Which items do you have?', 'What kinds of items are there?', 'What do you sell?'], 'responses': ['We sell coffee and tea', 'We have coffee and tea']}, {'tag': 'payments', 'patterns': ['Do you take credit cards?', 'Do you accept Mastercard?', 'Can I pay with Paypal?', 'Are you cash only?'], 'responses': ['Hey There!', 'How are you', 'We accept VISA, Mastercard and Paypal', 'We accept most major credit cards, and Paypal']}, {'tag': 'delivery', 'patterns': ['How long does delivery take?', 'How long does shipping take?', 'When do 